In [6]:
# ! pip install -q openai langchain langchain-text-splitters lxml octoai-sdk langchain-community faiss-cpu tiktoken transformers

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

url = "https://en.wikipedia.org/wiki/Star_Wars"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("div", "Divider")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

In [3]:
chunk_size = 1024
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Split
splits = text_splitter.split_documents(html_header_splits)

In [4]:
from langchain.vectorstores import FAISS

In [5]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="meta-llama-3-8b-instruct",
        max_tokens=1024,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
        
    )
embeddings = OctoAIEmbeddings()

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [6]:
vector_store = FAISS.from_documents(
    splits,
    embedding=embeddings
)

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
retriever = vector_store.as_retriever()

In [8]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
chain.invoke("Who is Luke's Father?")

: 

In [ ]:
from langchain.prompts import ChatPromptTemplate
template="""You are a literary critic. You are given some context and asked to answer questions based on only that context.
Question: {question} 
Context: {context} 
Answer:"""
lit_crit_prompt = ChatPromptTemplate.from_template(template)

In [1]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
lcchain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | lit_crit_prompt
    | llm
    | StrOutputParser()
)

NameError: name 'retriever' is not defined

In [15]:
from pprint import pprint

pprint(lcchain.invoke("What is the best thing about Luke's Father's story line?"))

(" The best thing about Luke's Father's story line is that it is a powerful "
 'and influential plot twist in cinema. It has strong repercussions on the '
 "saga and is a defining step of the hero's journey. It also symbolizes the "
 "theme of redemption, as Luke's father, Anakin, falls from grace and remains "
 'evil as Darth Vader until his redemption in Return of the Jedi. The story of '
 "Luke's father is a key part of the Star Wars saga and has had a lasting "
 'impact on popular culture. \n'
 'Note: The context provided is a collection of documents related to the Star '
 'Wars franchise, including information about the films, themes, and other '
 "media. The question asks about the best thing about Luke's Father's story "
 'line, which refers to the plot twist in which Luke discovers that Vader is '
 'his father. The answer is based on the information provided in the context '
 'and focuses on the significance and impact of this plot twist on the Star '
 'Wars saga. \n'
 "Source: